In [ ]:
pip install biopython

In [ ]:
from Bio import SeqIO
import numpy as np

In [ ]:
# ESTE SOLO ME BOTA LA MATRIZ DE SCORE
def crear_matriz(n,m):
  #creamos la matriz de ceros
  matriz = np.zeros((n+1,m+1))
  return matriz

def S(a,b):
  if (a == b):
    return 2
  elif ((a=="A" and b=="G") or (a=="C" and b=="T") or (a=="G" and b=="A") or (a=="T" and b=="C")):
    return -5
  else:
    return -7

def sw(seq1,seq2,gap):
  lens1=len(seq1)
  lens2=len(seq2)
  #creamos la matriz de ceros
  m_inicial=crear_matriz(lens1,lens2)
  punteros = crear_matriz(lens1,lens2)
  #Llenamos la primera fila y la primera columna de acuerdo al gap
  #m_inicial[:,0] = np.linspace(0,lens1*0,lens1 + 1)
  #m_inicial[0,:] = np.linspace(0,lens2*0,lens2 + 1)
  m_scores = m_inicial
  T=np.zeros(4) #T[0]:diagonal \, T[1]:arriba |, T[2]: izquierda <-- y T[3]:0
  for i in range(lens1):
    for j in range(lens2):
      T[0]= m_scores[i][j]+S(seq1[i],seq2[j])
      T[1]= m_scores[i,j+1] + gap
      T[2]= m_scores[i+1,j] + gap
      tmax = np.max(T)
      m_scores[i+1,j+1] = tmax
  print(m_scores) 

sw("AGC","AAG",-5) #3x4

In [ ]:
# AQUI ESTOY MODIFICANDO 
# ESTEEEEEEEEEEEEE FUNCIONA PERO SOLO ME BOTA UNA ALINEACION
def crear_matriz(n,m):
  #creamos la matriz de ceros
  matriz = np.zeros((n+1,m+1))
  return matriz

def S(a,b,S,identicalMatch,mismatch):
  if (S == True):
    if (a == b):
      return 2
    elif ((a=="A" and b=="G") or (a=="C" and b=="T") or (a=="G" and b=="A") or (a=="T" and b=="C")):
      return -5
    else:
      return -7
  else:
    match = identicalMatch
    difmatch = mismatch
    if (a == b):
      return match
    else:
      return difmatch
def finalize(align1, align2,Ss,identicalMatch,mismatch,gap):
    align1 = align1[::-1]    
    align2 = align2[::-1]    
    
    i,j = 0,0
    
    score = 0
    for i in range(0,len(align1)):
        # if two AAs are the same, then output the letter
        if align1[i] == align2[i]:                
            score += S(align1[i], align2[i],Ss,identicalMatch,mismatch)
    
        # if they are not identical and none of them is gap
        elif align1[i] != align2[i] and align1[i] != '-' and align2[i] != '-': 
            score += S(align1[i], align2[i],Ss,identicalMatch,mismatch)
    
        #if one of them is a gap, output a space
        elif align1[i] == "-" or align2[i] == "-":          
            score += gap
    
    print ("Score = ", score)
    print (align1)
    print (align2)

def sw(seq1,seq2,Ss,gapcost,identicalMatch=0,mismatch=-0):
  lens1=len(seq1)
  lens2=len(seq2)
  #creamos la matriz de ceros
  m_inicial=crear_matriz(lens1,lens2)
  punteros = crear_matriz(lens1,lens2)

  #Llenamos la primera fila y la primera columna de acuerdo al gap
  #m_inicial[:,0] = np.linspace(0,lens1*0,lens1 + 1)
  #m_inicial[0,:] = np.linspace(0,lens2*0,lens2 + 1)
  m_scores = m_inicial
  T=np.zeros(4) #T[0]:diagonal \, T[1]:arriba |, T[2]: izquierda <-- y T[3]:0
  for i in range(lens1):
    for j in range(lens2):
      T[0]= m_scores[i][j] + S(seq1[i],seq2[j],Ss,identicalMatch,mismatch)
      T[1]= m_scores[i][j+1] + gapcost
      T[2]= m_scores[i+1][j] + gapcost
      tmax = np.max(T)
      m_scores[i+1][j+1] = tmax
      if m_scores[i+1][j+1] == 0:
          punteros[i+1][j+1] = 0 # 0 means end of the path
      if m_scores[i+1][j+1] == T[2]: 
          punteros[i+1][j+1] = 3 # 3 means trace left
      if m_scores[i+1][j+1] == T[1]:
          punteros[i+1][j+1] = 2 # 2 means trace up
      if m_scores[i+1][j+1] == T[0]:
          punteros[i+1][j+1] = 1 # 1 means trace diagonal
           
  print("Matriz de scores") 
  print(m_scores)
  print("Matriz de Punteros")
  print(punteros)
  max_score = 0
  pos_max =np.zeros(2)
  for k in range(lens1+1):
    for l in range(lens2+1):
      if (m_scores[k][l]>= max_score):
        max_score=m_scores[k][l]
        pos_max[0]=k
        pos_max[1]=l
  
  align1=""
  align2=""
  i=int(pos_max[0])
  j=int(pos_max[1])
  # 1 : DIAGONAL- 2: up - 3: left
  while punteros[i][j] != 0:
    if punteros[i][j] == 1:
        align1 += seq1[i-1]
        align2 += seq2[j-1]
        i -= 1
        j -= 1
    elif punteros[i][j] == 2:
        align1 += '-'
        align2 += seq2[j-1]
        j -= 1
    elif punteros[i][j] == 3:
        align1 += seq1[i-1]
        align2 += '-'
        i -= 1 

  finalize(align1,align2,Ss,identicalMatch,mismatch,gapcost)  


In [ ]:
s1="AGC"
s2="AAG"
# parametros:sw(seq1,seq2,Ss,gapcost,identicalMatch=-1,mismatch=-1)
# es True si hay matriz de similitud
# es False si no hay matriz de similitud
sw(s1,s2,True,-5) #3x4

In [ ]:
sequences = SeqIO.parse("Q5H8A6.fasta", "fasta")
for record in sequences:
   data1 = str(record.seq.upper()) # the fasta file just have one sequence
sequences = SeqIO.parse("Q41131.fasta", "fasta")
for record in sequences:
   data2 = str(record.seq.upper()) # the fasta file just have one sequence

# parametros:sw(seq1,seq2,Ss,gapcost,identicalMatch=-1,mismatch=-1)
# es True si hay matriz de similitud
# es False si no hay matriz de similitud
sw(data1,data2,False,-5,2,-2) #3x4

In [ ]:
sequences = SeqIO.parse("Q5H8A6.fasta", "fasta")
for record in sequences:
   data1 = str(record.seq.upper()) # the fasta file just have one sequence
sequences = SeqIO.parse("Q41131.fasta", "fasta")
for record in sequences:
   data2 = str(record.seq.upper()) # the fasta file just have one sequence

# parametros:sw(seq1,seq2,Ss,gapcost,identicalMatch=-1,mismatch=-1)
# es True si hay matriz de similitud
# es False si no hay matriz de similitud
sw(data1,data2,False,-2,1,-1) #3x4

In [ ]:
sequences = SeqIO.parse("Q5H8A6.fasta", "fasta")
for record in sequences:
   data1 = str(record.seq.upper()) # the fasta file just have one sequence
sequences = SeqIO.parse("Q41131.fasta", "fasta")
for record in sequences:
   data2 = str(record.seq.upper()) # the fasta file just have one sequence

# parametros:sw(seq1,seq2,Ss,gapcost,identicalMatch=-1,mismatch=-1)
# es True si hay matriz de similitud
# es False si no hay matriz de similitud
sw(data1,data2,False,-1,1,-2) #3x4